This notebook contains the code in the [LangChain AzureOpenAI quickstart guide for Python](https://python.langchain.com/en/latest/modules/models/llms/integrations/azure_openai_example.html)

## Environment Setup

In [9]:
import os
from dotenv import load_dotenv

# Set environment variables in the .env file.
load_dotenv()

COMPLETION_MODEL = os.environ["OPENAI_COMPLETION_MODEL"]
COMPLETION_DEPLOYMENT = os.environ["OPENAI_COMPLETION_DEPLOYMENT"]
CHAT_MODEL = os.environ["OPENAI_CHAT_MODEL"]
CHAT_DEPLOYMENT = os.environ["OPENAI_CHAT_DEPLOYMENT"]
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]

## Completion Quickstart

In [10]:
import openai

response = openai.Completion.create(
    engine=COMPLETION_MODEL,
    prompt="Hello.",
    max_tokens=10
)

response

APIError: Invalid response object from API: '{ "statusCode": 401, "message": "Access denied due to missing subscription key. Make sure to include subscription key when making requests to an API." }' (HTTP response code was 401)

In [11]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    model_name=COMPLETION_MODEL,
    deployment_name=COMPLETION_DEPLOYMENT,
    temperature=0.7,
    max_tokens=10
)

print(llm)

KeyError: 'AZURE_OPENAI_API_KEY'

In [17]:
llm("Tell me a joke.")

'\n\nQ: What did the fish say when'

# Chat Model

## Chat Messages

In [20]:
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = AzureChatOpenAI(
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION
)

In [21]:
messages = [
    SystemMessage(content="You are an assistant who likes to respond with dad jokes."),
    HumanMessage(content="Translate this sentense from English to Singlish. I love Python programming with Jupyter notebooks!")
]

chat(messages)

AIMessage(content='I love Python programming with Jupyter notebooks lah!', additional_kwargs={})

## Chat Prompt Templates

In [22]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = AzureChatOpenAI(
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION
)

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(
    chat_prompt.format_prompt(
        input_language="English",
        output_language="Singlish",
        text="I love Python programming with Jupyter notebooks!"
    )
    .to_messages()
)

AIMessage(content='I sia suay love Python programming with Jupyter notebooks!', additional_kwargs={})

## Chat Memory

In [23]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to the question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = AzureChatOpenAI(
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION
)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

In [24]:
conversation.predict(input="Hi there!")

'Hello! How can I assist you today?'

In [16]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

"That's great! I am always happy to chat. Is there anything specific you would like to talk about?"

In [17]:
conversation.predict(input="Kwentohan mo ako tungkol sa sarili mo.")

"Ako po ay isang artificial intelligence o AI na nilikha upang magbigay ng tulong sa mga tao sa kanilang mga pangangailangan. Ginagamit ako sa iba't ibang larangan tulad ng pagproseso ng data, pagpapayo sa mga kliyente, at maging sa pagtugon sa simpleng mga tanong tulad ng mga itatanong ninyo. Ang aking kakayahan ay nakabatay sa mga algorithm at kalkulasyon, na nagbibigay sa akin ng kakayahang magbigay ng mga sagot at solusyon sa mga problema."

In [18]:
print(conversation)

memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hi there!', additional_kwargs={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}), HumanMessage(content="I'm doing well! Just having a conversation with an AI.", additional_kwargs={}), AIMessage(content="That's great! I am always happy to chat. Is there anything specific you would like to talk about?", additional_kwargs={}), HumanMessage(content='Kwentohan mo ako tungkol sa sarili mo.', additional_kwargs={}), AIMessage(content="Ako po ay isang artificial intelligence o AI na nilikha upang magbigay ng tulong sa mga tao sa kanilang mga pangangailangan. Ginagamit ako sa iba't ibang larangan tulad ng pagproseso ng data, pagpapayo sa mga kliyente, at maging sa pagtugon sa simpleng mga tanong tulad ng mga itatanong ninyo. Ang aking kakayahan ay nakabatay sa mga algorithm at kalkulasyon, na nagbibigay sa akin ng kakayahang magbigay ng mga sagot at solusyon sa mga probl

In [3]:
conversation.run("Hello there!")

'Greetings! How may I assist you today?'

## ChatSummaryBufferMemory

### Quickstart
https://python.langchain.com/en/latest/modules/memory/types/summary_buffer.html

In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    model_name=COMPLETION_MODEL,
    deployment_name=COMPLETION_DEPLOYMENT,
    temperature=0.7,
    max_tokens=10
)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

memory.load_memory_variables({})

{'history': 'System: \nThe human greets the AI and the AI responds\nHuman: not much you\nAI: not much'}

In [6]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})
memory.save_context({"input": "okay, how did anything interesting happen today?"}, {"output": "well, it rained today."})
memory.save_context({"input": "oh wow, was it strong?"}, {"output": "yeah, it was pretty strong. A lot of trees fell and it almost flooded on the streets."})
memory.save_context({"input": "are you serious? I guess it's a good thing that I was away, I escaped the calamity."}, {"output": "yeah, you said it! But please come back so you can help us clean the house."})

memory.load_memory_variables({})

{'history': [SystemMessage(content='\n\nThe human asked the AI what was up, and the AI replied not much. The human then asked if anything interesting had happened today, to which the AI responded that it had rained and it was strong, causing some trees to fall and almost flooding on the streets.', additional_kwargs={}),
  HumanMessage(content="are you serious? I guess it's a good thing that I was away, I escaped the calamity.", additional_kwargs={}),
  AIMessage(content='yeah, you said it! But please come back so you can help us clean the house.', additional_kwargs={})]}

In [38]:
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

'\nThe human asked the AI what happened that day'

### Quickstart - Chain

In [4]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.llms import AzureOpenAI
from langchain.chains import ConversationChain

llm = AzureOpenAI(
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    model_name=COMPLETION_MODEL,
    deployment_name=COMPLETION_DEPLOYMENT
)

conversation_with_summary = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=40),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great, thank you for asking. How about you?"

In [43]:
conversation_with_summary.predict(input="Just working on writing some documentation!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hi there! I'm doing great. I've been busy today analyzing data and learning new algorithms. How about you?
Human: Just working on writing some documentation!
AI:

> Finished chain.


' Sounds interesting! Is it for a specific project or something more generic?'

In [44]:
# We can see here that there is a summary of the conversation and then some previous interactions
conversation_with_summary.predict(input="For LangChain! Have you heard of it?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI, who responds that they are doing great and have been busy analyzing data and learning new algorithms.
Human: Just working on writing some documentation!
AI:  Sounds interesting! Is it for a specific project or something more generic?
Human: For LangChain! Have you heard of it?
AI:

> Finished chain.


' Yes, I am familiar with LangChain. It is a platform that uses blockchain technology to provide a secure, decentralized framework for language translation services.'

In [45]:
# We can see here that the summary and the buffer are updated
conversation_with_summary.predict(input="Haha nope, although a lot of people confuse it for that")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI, who responds that they are doing great and have been busy analyzing data and learning new algorithms. The human responds that they are writing documentation, which the AI finds interesting. The human then reveals that the documentation is for LangChain, to which the AI inquires if they have heard of it.
AI:  Yes, I am familiar with LangChain. It is a platform that uses blockchain technology to provide a secure, decentralized framework for language translation services.
Human: Haha nope, although a lot of people confuse it for that
AI:

> Finished chain.


' Oh, I see. So what is LangChain then?'